In [ ]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

load_dotenv()


In [ ]:
MODEL = "gpt-3.5-turbo"

OUTPUT_FOLDER = "../generated_GUIs/"

CREATION_PROMPT = PromptTemplate(
    input_variables=["summary"],
    template= """create the HTML code for a Mobilepage, 
    with a bit of styling, that represents following functionality: 
    {summary}."""
)

SUMMARY_PROMPT = PromptTemplate(
    input_variables=["sum1","sum2","sum3","sum4","sum5"],
    template=
        """Given a collection on Screensummaries, which descripe the same screen:
        Summary 1: {sum1}.
        Summary 2: {sum2}.
        Summary 3: {sum3}.
        Summary 4: {sum4}.
        Summary 5: {sum5}.

        Extract the provided information of the screen and formulate a new description."""
)

In [ ]:
#function for Creating a GUI based on a given description
def create_gui(id, summary, prompt=CREATION_PROMPT, temp=0):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)

    chain = LLMChain(llm=llm, prompt=prompt)

    try: 
        response = chain.run(summary=summary)
    except Exception as e:
        print(e)
    
    print(response)
    with open(OUTPUT_FOLDER + f"{id}.html", 'w') as f:
        f.write(response)


In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser

#Sequential chain, of summarizing the Screen2Words Summaries into one and creating a GUI
def create_gui_S2W(id, sum1, sum2, sum3, sum4, sum5, temp=0):
    llm = ChatOpenAI(model_name=MODEL, temperature=temp)
    
    summary_chain = SUMMARY_PROMPT | llm | StrOutputParser()
    creation_chain = CREATION_PROMPT | llm | StrOutputParser()
    
    chain = {"summary": summary_chain} | RunnablePassthrough.assign(creation=creation_chain)
    
    try: 
        response = chain.invoke({"sum1": sum1,"sum2": sum2,"sum3": sum3,"sum4": sum4,"sum5": sum5})
    except Exception as e:
        print(e)
    
    print(response)
    with open(OUTPUT_FOLDER + f"{id}_S2W.html", 'w') as f:
        f.write(response.get('creation'))
    return response

In [ ]:
SUM1 = "page showing search option in a flight booking app"
SUM2 = "screen displaying multiple options in an airline booking application"
SUM3 = "screen showing with search flights"
SUM4 = "search page for a flight booking application"
SUM5 = "searching flights in a transport app"

response = create_gui_S2W(377, SUM1,SUM2,SUM3,SUM4,SUM5)

In [ ]:
DESCRIPTION = "This is a flight booking screen from a travel app where users can search for flights, specifying departure and arrival cities, travel dates, number of passengers, and class of service. It also offers options for round trip or one-way flights and displays a section for recent searches"

create_gui(377, DESCRIPTION)

In [ ]:
#test